# Imports

In [ ]:
import torch, numpy as np

from torch.utils.data import DataLoader

from datasets import Dataset, load_dataset

import IPython
import IPython.display
import julius
import librosa

import time


import torchaudio
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps


import os
import uuid


import whisper
import os


# Load datasets

[ivrit.ai](https://huggingface.co/datasets/ivrit-ai/crowd-transcribe-v5) </br>
[HebDB](https://huggingface.co/datasets/SLPRL-HUJI/HebDB) </br>

Might use [Hebrew speech Kan](https://huggingface.co/datasets/imvladikon/hebrew_speech_kan) and [Shaul's dataset](https://openslr.org/134/)


In [ ]:
data = load_dataset("SLPRL-HUJI/HebDB", "DK_raw", streaming=True)
print(data)

In [ ]:
torch_train = data['train'].with_format("torch")
# torch_test = data['test'].with_format("torch")

for i in torch_train:
    res = i
    break

res

# Resample to 16KHz
They did this using julius, librosa runs 200x faster and sounds the fucking same </br>
Julius also sucks!! This is straight shit!! I do not know why are there so many methods, and which one to use. </br></br>
Librosa is more relatable like "yo bro you wanna resample? I gotchu!!! use my <b>RESAMPLE</b> method". </br>
Julius is a dickhead it's like "Oh yeah you should analyze which specific function from within my 40000 modules you want".

In [ ]:
audio = res['audio']['array']
sr = res['audio']['sampling_rate'].item()

In [ ]:
bef = time.time()

res_julius = julius.resample._downsample2(audio, 16000)

aft = time.time()

print('Resampling took ' + str(aft-bef) + ' seconds')
IPython.display.Audio(res_julius, rate=16000)

In [ ]:
bef = time.time()

res_librosa = librosa.resample(audio.detach().numpy(), orig_sr=sr, target_sr=16000)

aft = time.time()

print('Resampling took ' + str(aft-bef) + ' seconds')
IPython.display.Audio(res_librosa, rate=16000)

# Perform VAD and split data to short segments

In [ ]:
vad_model = load_silero_vad()

speech_timestamps = get_speech_timestamps(
  res_librosa,
  vad_model,
  return_seconds=True,
)

speech_timestamps

In [ ]:
MIN_DURATION = 1.5  # seconds
SILENCE_GAP = 0.1   # 100 ms
PAD = 0.03          # 30 ms

sr = 16000

valid_segments = []
for seg in speech_timestamps:
    duration = seg['end'] - seg['start']

    if duration >= MIN_DURATION:
        valid_segments.append(seg)

# You also may want to merge segments that are closer than SILENCE_GAP
merged_segments = []
for seg in valid_segments:
    if not merged_segments:
        merged_segments.append(seg)
    else:
        prev = merged_segments[-1]
        # If current segment starts within the SILENCE_GAP from the previous segment’s end, merge them
        if seg['start'] - prev['end'] < SILENCE_GAP:
            merged_segments[-1]['end'] = seg['end']
        else:
            merged_segments.append(seg)


merged_segments

In [ ]:
print(f'{len(merged_segments)=}')
lens = np.array([i['end']-i['start'] for i in merged_segments]).mean()
print(f'Mean length is {lens}')

In [ ]:
MIN_ACTIVITY_FOR_ENROLL = 3

pad_samples = int(PAD * sr)  # Number of samples for 30ms
silence = torch.zeros(pad_samples)  # Shape: (1, pad_samples) for mono

output_dir = "DK_samples_1"
os.makedirs(output_dir, exist_ok=True)


whisper_model = whisper.load_model("large-v2")
vad_model = load_silero_vad()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
whisper_model.to(device)

for i, seg in enumerate(merged_segments):
    start_sample = int(seg['start'] * sr)
    end_sample   = int(seg['end'] * sr)

    segment_waveform = torch.cat((silence, torch.tensor(res_librosa[start_sample:end_sample]), silence))

    previous_4_sec = res_librosa[start_sample-(sr*4) : start_sample]

    prev_activity = get_speech_timestamps(
        previous_4_sec,
        vad_model,
        return_seconds=True
        )
    speech_time_in_prev_4_sec = np.array([j['end']-j['start'] for j in prev_activity]).sum()

    # We want this segment only if we have an enrollment for it
    if speech_time_in_prev_4_sec >= MIN_ACTIVITY_FOR_ENROLL:

        while segment_waveform.ndim > 1:
            segment_waveform = segment_waveform.squeeze(0)

        result = whisper_model.transcribe(segment_waveform, language="he")  # specify Hebrew if needed

        with open('data.txt', 'a') as output_data:
            output_data.write(output_dir+f'/target_{i}.wav'+'\t<SEP>\t'+result['text']+'\t<SEP>\t'+output_dir+f'/enrollment_{i}.wav')


        # Save target audio to synthesize, and the enrollment
        if segment_waveform.ndim == 1:
            segment_waveform = segment_waveform.unsqueeze(0)
        torchaudio.save(output_dir+f'/target_{i}.wav', segment_waveform, sr)

        enroll = torch.cat((silence, torch.tensor(previous_4_sec), silence))
        if enroll.ndim == 1:
            enroll = enroll.unsqueeze(0)

        torchaudio.save(output_dir+f'/enrollment_{i}.wav', enroll, sr)

    else:
        continue

# Transcribe each segment using Whisper V2 large
This is a large model (1.5B params) but we can run it now uni servers. We only need some inference. </br>
On my CPU (12th gen intel i5-12400) inference took $\approx45$ sec per audio segment ($\approx12$ sec audio), so maybe we have hope of finishing this shit quickly on Uni GPUs!

In [ ]:
model = whisper.load_model("large-v2")  # loads the Whisper Large V2 model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
device

In [ ]:
segmented_dir = "segmented_wav"

results = []
j=0

for filename in os.listdir(segmented_dir):
    if filename.endswith(".wav"):
        audio_path = os.path.join(segmented_dir, filename)

        audio, sr = librosa.load(audio_path, sr=16000, mono=True)

        result = model.transcribe(audio, language="he")  # specify Hebrew if needed

        results.append({filename : result['text']})

        j+=1
        print(f'Sample {j} fin')
        if j == 20: break


In [ ]:
for res in results:
    print(res, end = 3*'\n')